In [1]:
from PIL import Image
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import scipy.misc

In [2]:
onlyfiles = [f for f in listdir('./Datasets') if isfile(join('./Datasets', f))]
onlyfiles.remove('.DS_Store')
print('Totally: ',len(onlyfiles))

Totally:  28


In [3]:
onlyfiles.sort()

In [4]:
onlyfiles

['01. 185x354.TIF',
 '02. 202x216.TIF',
 '03. 272x369.TIF',
 '04. 363x189.tiff',
 '05. 379x281.tiff',
 '06. 197x168.tiff',
 '07. 243x413 .tiff',
 '08. 329x232.tif',
 '09. 279x187.tif',
 '10. 269x227.TIF',
 '11. 394x248.jpg',
 '12. 305x492.jpg',
 '13. 206x184.png',
 '14. 279x152.png',
 '15. 279x152.png',
 '16. 335x476.jpg',
 '17. 283x300.jpg',
 '18. 217x278.png',
 '19. 283x191.jpg',
 '20. 135x390.tif',
 '21. 203x258.jpg',
 '22. 257x375.jpg',
 '23. 84x68.png',
 '24. 203x224.tif',
 '25. 156x115.tif',
 '26. 197x187.tif',
 '27. 278x419.tiff',
 '28. 267x264.png']

In [5]:
def adaptive_choose_threshold(img, q=90):
    
    dim = np.array(img).shape # array
    hori_lim = dim[1] - 224 # Horizontal corresponds to cols
    vert_lim = dim[0] - 224 # Vertical corresponds to rows
    
    hori_rand = np.random.randint(low=0, high=hori_lim, dtype=np.int)
    vert_rand = np.random.randint(low=0, high=vert_lim, dtype=np.int)
    
    x0 = np.zeros(5000)
    for i in range(5000):
        hori_rand = np.random.randint(low=0, high=hori_lim, dtype=np.int)
        vert_rand = np.random.randint(low=0, high=vert_lim, dtype=np.int)
        crop_pic = img.crop((hori_rand, vert_rand, hori_rand+224, vert_rand+224))
        x0[i] = np.var(crop_pic)
    return np.percentile(x0, q)

In [9]:
def sample_from_pic(load_path, samples=200, save_name=None, q_thresh=90):
    
    size = 4 * samples
    
    x = np.zeros((size, 224, 224, 3))
    
    img = Image.open(load_path) # pil instance
    thresh = adaptive_choose_threshold(img, q=q_thresh)
    print('Var Threshold: ', thresh)
    
    dim = np.array(img).shape # array
    hori_lim = dim[1] - 224 # Horizontal corresponds to cols
    vert_lim = dim[0] - 224 # Vertical corresponds to rows
    
    t = 0
    while(t < size):
        hori_rand = np.random.randint(low=0, high=hori_lim, dtype=np.int)
        vert_rand = np.random.randint(low=0, high=vert_lim, dtype=np.int)
        crop_pic = img.crop((hori_rand, vert_rand, hori_rand+224, vert_rand+224))
        if(np.var(np.array(crop_pic)) <= thresh ):
            continue            
        crop_pic = crop_pic.convert(mode='RGB')
        x[t] = np.array(crop_pic) 
        t += 1
    var = np.var(x, axis=(1,2,3))
    x = x[var.argsort()] # var from small to large
    x = x[-samples:]
    
    ## save crop pics selectly
    if(save_name != None):
        save_idx = np.linspace(start=0, stop=samples-1, num = 20, dtype=np.int)
        x_save = x[save_idx]
        for i in range(10):
            scipy.misc.imsave('./Crop/CropShow/' + save_name + str(i+1).zfill(2) + '.jpg', x_save[i])

    return x

In [10]:
size = 200
all_crop_pics = np.zeros((size*28, 224, 224, 3))

for idx, name in enumerate(onlyfiles):
    load_path = join('./Datasets', name)
    x = sample_from_pic(load_path, samples=size, save_name=str(idx+1).zfill(2) + '-pic-',\
                        q_thresh = 45)
    all_crop_pics[size*idx:size*(idx+1)] = x
    print('Finish: ', (idx+1))

Var Threshold:  980.501663756
Finish:  1
Var Threshold:  1181.69352916
Finish:  2
Var Threshold:  985.827531461
Finish:  3
Var Threshold:  1844.39967222
Finish:  4
Var Threshold:  1193.70797562
Finish:  5
Var Threshold:  657.805554655
Finish:  6
Var Threshold:  3670.21080765
Finish:  7
Var Threshold:  2038.5997432
Finish:  8
Var Threshold:  524.312232705
Finish:  9
Var Threshold:  2208.34852628
Finish:  10
Var Threshold:  846.599314484
Finish:  11
Var Threshold:  1527.43212902
Finish:  12
Var Threshold:  1505.77916971
Finish:  13
Var Threshold:  1778.90119112
Finish:  14
Var Threshold:  2843.81368924
Finish:  15
Var Threshold:  1954.37798129
Finish:  16
Var Threshold:  1465.84919453
Finish:  17
Var Threshold:  2902.87255734
Finish:  18
Var Threshold:  1085.89670941
Finish:  19
Var Threshold:  2855.18882133
Finish:  20
Var Threshold:  322.344523814
Finish:  21
Var Threshold:  756.28614677
Finish:  22
Var Threshold:  2684.70137958
Finish:  23
Var Threshold:  1915.10581935
Finish:  24
Var

In [11]:
for i in range(28):
    np.save('./Crop/'+'pic'+str(i+1)+'.npy', all_crop_pics[size*i:size*(i+1)])
    print('Finish: ', i)

Finish:  0
Finish:  1
Finish:  2
Finish:  3
Finish:  4
Finish:  5
Finish:  6
Finish:  7
Finish:  8
Finish:  9
Finish:  10
Finish:  11
Finish:  12
Finish:  13
Finish:  14
Finish:  15
Finish:  16
Finish:  17
Finish:  18
Finish:  19
Finish:  20
Finish:  21
Finish:  22
Finish:  23
Finish:  24
Finish:  25
Finish:  26
Finish:  27
